##Zadanie 1.
Zaimplementuj w środowisku MPI obliczanie równoległe liczby PI z wzoru Leibniz-a.
Proces 0, to proces Root, który zarządza obliczeniami:
- wysyła do procesów roboczych Slave, indeksy wyrazów szeregu poczatek_local i koniec_local, 
z których należy obliczyć wyniki cząstkowe
- zbiera wyniki cząstkowe z procesów i wyświetla wynik przybliżenia PI

In [ ]:
%%sh
cat > pi-mpi.c <<EOF
#include 
#include 
#include 

// Leibniz 

double potega(double podstawa , int wykladnik ){
  	double wynik=1.0;
  for (int i = 1; i <= wykladnik; i++)
    {
        wynik*=podstawa;
    }


  return wynik ; 
}

int main(int argc, char *argv[])
{
    //tworzenie zmiennych globalnych
    int i,np,lp;
    int tag=50;
    MPI_Status status;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &np);
    MPI_Comm_size(MPI_COMM_WORLD, &lp);
    
    //jesli proces ma numer 0 to:
    if(np ==0)
    {
      //tworzenie zmiennych lokalnych
      double pole = 0 ,s=0;
      printf("\n Obliczenie liczny pi ze wzoru leibniza");
   
      //odbieranie oraz sumowanie wynikow z wszystkich procesow

      for(i=1; i<lp; i++)
      {
        MPI_Recv(&s, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        pole +=s;
      }

      //wypisanie wartosci pola
      printf("\nwartosc liczby pi ze wzoru Leibnitza = %1f\n", 4 *  pole);
    }

    if(np != 0)
    {
      //tworzenie zmiennych lokalnych
      double s1 = potega( -1 , np - 1   )   / ( 2 *  np - 1  )   ; 
      printf("%f :  " , s1);
      // s1 = 0;
      //wysylanie liczby 
      MPI_Send(&s1, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
    }
    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 40 --allow-run-as-root a.out

##Zadanie 2
Zaimplementuj w środowisku MPI wyznaczenie numerycznej wartość całki y=f(x) (postać funkcji 
wybierasz sam) w przedziale <a,b> przy pomocy N trapezów.
Proces 0, to proces Root, który zarządza obliczeniami:
- wysyła do procesów roboczych Slave początek a_local i koniec b_local lokalnego przedziału 
całkowania dla danego procesu oraz liczbę N_local trapezów, z których należy policzyć całkę
- zbiera wyniki cząstkowe z procesów, i wyświetla wynik całki

In [ ]:
%%sh 
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
double funkcja(double x) {
return x; 
}
int main(int argc, char *argv[]) {
int i,np,lp;
int tag=50;
MPI_Status status;
MPI_Init(&argc, &argv);
MPI_Comm_rank(MPI_COMM_WORLD, &np);
MPI_Comm_size(MPI_COMM_WORLD, &lp);
//jeśli proces ma numer 0 to:
if (np == 0) {
//Tworzenie zmiennych lokalnych
double h,xp,xk,pole,s=0;
printf("\nObliczanie calki oznaczonej metoda trapezu dla funkcji f(x)=x");
//Poczatek przedzialu calkowania xp 
xp = 1.0;
//Koniec przedzialu calkowania xk 
xk = 4.0;
// obliczanie wartości h
h=(xk-xp)/(double)lp;
// wysylanie wartosci ci xp i xk oraz h do poszczegolnych procesow
for(i=0;i<lp;i++)
{
MPI_Send(&xp, 1, MPI_DOUBLE, i,tag, MPI_COMM_WORLD);
MPI_Send(&xk, 1, MPI_DOUBLE, i,tag, MPI_COMM_WORLD);
MPI_Send(&h, 1, MPI_DOUBLE, i,tag, MPI_COMM_WORLD);
}
// odbieranie oraz sumowanie wyników z wszystkich procesów 
for(i=0;i<lp;i++)
{
MPI_Recv(&s, 1, MPI_DOUBLE, i, tag,MPI_COMM_WORLD, &status);
pole+=s;
printf("Pole trapezu wynosi: %1f\n"  , s) ;

}
// wypisanie wartości pola
printf("\nPole pod calka wynosi = %lf\n",pole); 
}
else
{
//tworzenie zmiennych lokalnych
double s,h,xp,xk;
double np2=np;
// odbieranie wartości od procesu 0 
MPI_Recv(&xp, 1, MPI_DOUBLE, 0, tag,MPI_COMM_WORLD, &status);
MPI_Recv(&xk, 1, MPI_DOUBLE, 0, tag,MPI_COMM_WORLD, &status);
MPI_Recv(&h, 1, MPI_DOUBLE, 0, tag,MPI_COMM_WORLD, &status);
// obliczanie pola trapezu dla podanej funkcji
s=( funkcja(  xp  +  (np - 1 )  * h)  + funkcja(  xp  +  np * h)  ) * 0.5 * h; 
//wysyłanie pola trapezu do procesu 0
MPI_Send(&s, 1, MPI_DOUBLE, 0,tag, MPI_COMM_WORLD); 
}
MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 8 --allow-run-as-root a.out



Obliczanie calki oznaczonej metoda trapezu dla funkcji f(x)=xPole trapezu wynosi: 1.000000
Pole trapezu wynosi: 0.445312
Pole trapezu wynosi: 0.585938
Pole trapezu wynosi: 0.726562
Pole trapezu wynosi: 0.867188
Pole trapezu wynosi: 1.007812
Pole trapezu wynosi: 1.148438
Pole trapezu wynosi: 1.289062

Pole pod calka wynosi = 7.070312


##Zadanie 3
Zaimplementuj program mnożenia macierzy A przez wektor B
Proces 0, to proces Root, który zarządza obliczeniami:
- inicjuje wartości macierzy A i wektora B 
- wysyła do procesów roboczych Slave fragmenty macierzy A i wektora B 
- zbiera wyniki cząstkowe z procesów, i wyświetla wynik całki

In [ ]:
%%sh
cat > pi-mpi.c << EOF
#include <stdlib.h> #include <stdio.h> #include <mpi.h> #include <time.h> #include <sys/time.h>
/* Przyjmujemy, ze proces 0 to proces Root, który rozdziela wiersze
i kolumny macierzy B, C pomiędzy procesy robocze Slave wykonujące 
obliczenia cząstkowe mnożenia macierzy. Procesy Slave zwracają wyniki 
do procesu Root, z których składa macierz wynikową A */
//Liczba wierszy i kolumn w macierzach A, B, C #define N 40
MPI_Status status;
double A[N][N],B[N],C[N][N];
int main(int argc, char **argv) {
int processCount, processId, slaveTaskCount, source, dest, rows, offset;
struct timeval start, stop;
//Inicjalizacja srodowiska MPI
MPI_Init(&argc, &argv);
//Uzyskanie numeru aktualnego procesu
MPI_Comm_rank(MPI_COMM_WORLD, &processId);
//Uzyskanie liczby wszystkich procesów
MPI_Comm_size(MPI_COMM_WORLD, &processCount);
//Liczba procesów Slave mniejsza o 1 niz processCount
slaveTaskCount = processCount - 1;
//Przyjęto, ze proces 0 Root (Master) - poniżej jego kod
if (processId == 0) {
double start = MPI_Wtime();
// Inicjalizacja macierzy A i B
srand ( time(NULL) );
for (int i = 0; i<N; i++) {
for (int j = 0; j<N; j++) {
A[i][j]= rand()%10;
//B[i][j]= rand()%10;
}}
//Wypisanie zawartości macierzy A i B
printf("\n Mnozenie macierzy za pomoca MPI \n");
printf("\nMacierz A\n\n");
for (int i = 0; i<N; i++) {
for (int j = 0; j<N; j++) {
printf("%.0f\t", A[i][j]);
}
printf("\n");
}
printf("\nMacierz B\n\n");
for (int i = 0; i<N; i++) {
for (int j = 0; j<N; j++) {
printf("%.0f\t", B[i][j]);
}
printf("\n");
}
//Okreslene liczby wierzy macierzy A, która zostanie wysłana o każdego
//z procesów Slave
rows = N/slaveTaskCount;
//Zmienna offset określa aktualny pierwszy z wierszy do wysłania do
//aktualnego procesu Slave
offset = 0;
// Przygotowujemy wiersze i kolumny do wysłania do kolejnych procesów
//Slave o Id od 1 do slaveTaskCount, zostaną one przesłane wiadomością 
//z tag 1
for (dest=1; dest <= slaveTaskCount; dest++)
{
//Przesylamy offset wzledem wiersza 0
MPI_Send(&offset, 1, MPI_INT, dest, 1, MPI_COMM_WORLD);
//Ile wierszy przesylamy
MPI_Send(&rows, 1, MPI_INT, dest, 1, MPI_COMM_WORLD);
//przesylamy wiersze macierzy A do procesow Slave
MPI_Send(&A[offset][0], rows*N, MPI_DOUBLE,dest,1, MPI_COMM_WORLD);
//Przesylamy kolumny macierzy B do procesow Slave
MPI_Send(&B, N*N, MPI_DOUBLE, dest, 1, MPI_COMM_WORLD);
//Modyfikujemy aktualną wartość zmiennej offset
offset = offset + rows; }
//Proces Root czeka aż procesy Slave obliczą cząstkowe iloczyny wierszy
//i kolumn z macierzy A i B, wyniki zostaną odebrane z wiadomościach
// z tag 2
for (int i = 1; i <= slaveTaskCount; i++)
{
source = i;
//Proces Root otrzymuje offset od aktualnego procesu Slave
MPI_Recv(&offset, 1, MPI_INT, source, 2, MPI_COMM_WORLD, &status);
//Proces Root otrzymuje liczbę wierszy, którą otrzyma od aktualnego
//procesu Slave
MPI_Recv(&rows, 1, MPI_INT, source, 2, MPI_COMM_WORLD, &status);
//Otrzymanie danych cząstkowych z mnożenia A*B do C
MPI_Recv(&C[offset][0], rows*N, MPI_DOUBLE, source, 2 , MPI_COMM_WORLD, &status);
}
//Wypisanie macierzy wynikowej C
printf("\nWynikowa macierz C = A * B:\n\n");
for (int i = 0; i<N; i++) {
for (int j = 0; j<N; j++)
printf("%.0f\t", C[i][j]);
printf ("\n");
}
printf ("\n");
double end = MPI_Wtime();
printf("Czas obliczen %f",end - start);
} //Koniec kodu procesu Root
// Kod do wykonania przez procesy Slave
if (processId > 0) {
// Podanie procesom Slave numeru procesu Root
source = 0;
//Procesy Slave czekają na wiadomości z tag 1 od procesu Root
//Każdy z procesów Slave wykonuje oddzielnie następujący kod
//Procesy Slave otrzymują wartość offsetu od procesu Root
MPI_Recv(&offset, 1, MPI_INT, source, 1, MPI_COMM_WORLD, &status);
// Procesy Slave otrzymują liczbę wierszy, która zostanie przesłana od
// procesu Root
MPI_Recv(&rows, 1, MPI_INT, source, 1, MPI_COMM_WORLD, &status);
// Procesy Slave otrzymują fragmenty macierzy A od procesu Root
MPI_Recv(&A, rows*N, MPI_DOUBLE, source, 1, MPI_COMM_WORLD, &status);
// Procesy Slave otrzymują fragmenty macierzy B od procesu Root
MPI_Recv(&B, N*N, MPI_DOUBLE, source, 1, MPI_COMM_WORLD, &status);
// Mnożenie macierz y
for (int k = 0; k<N; k++) {
for (int i = 0; i<rows; i++) {
//
C[i][k] = 0.0;
// Element A[i][j] mnożony przez B[j][k]
for (int j = 0; j<N; j++)
C[i][k] = C[i][k] + A[i][j] * B[j][k];
}}
// Wyniki cząstkowe są wysyłane z procesów Slave do procesu
Root wiadomościami z tag 2
//Offset zostanie wysłany do Root
MPI_Send(&offset, 1, MPI_INT, 0, 2, MPI_COMM_WORLD);
//Liczba wierszy zostanie wysłana do Root
MPI_Send(&rows, 1, MPI_INT, 0, 2, MPI_COMM_WORLD);
//Wyznaczony fragment macierzy C zostanie wysłany do Root
MPI_Send(&C, rows*N, MPI_DOUBLE, 0, 2, MPI_COMM_WORLD);
}
MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 4 --allow-run-as-root a.out